In [44]:
from bs4 import BeautifulSoup
import os
import requests
import urllib
import re
import pandas as pd
import numpy as np

In [6]:
records = pd.read_csv('records.csv', index_col=0)

In [182]:
for i in range(1, 5):
    records['color{}'.format(i)] = np.zeros(360)


In [183]:
records

,team_name,wins,losses,w/l_ratio,most_recent_elo,team_city,team_state,team_id,color1,color2,color3,color4
0,Abilene Chr,78,106,0.423913,687.653203,Abilene,TX,1101,0.0,0.0,0.0,0.0
1,Air Force,243,278,0.466411,668.757677,USAF Academy,CO,1102,0.0,0.0,0.0,0.0
2,Akron,364,198,0.647687,793.686137,Akron,OH,1103,0.0,0.0,0.0,0.0
3,Alabama,324,238,0.576512,801.734237,Tuscaloosa,AL,1104,0.0,0.0,0.0,0.0
4,Alabama A&M,151,340,0.307536,485.358751,Normal,AL,1105,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
355,Wright St,309,236,0.566972,737.454923,Dayton,OH,1460,0.0,0.0,0.0,0.0
356,Wyoming,252,285,0.469274,665.159126,Laramie,WY,1461,0.0,0.0,0.0,0.0
357,Xavier,401,175,0.696181,834.066729,Cincinnati,OH,1462,0.0,0.0,0.0,0.0
358,Yale,270,228,0.542169,778.892366,New Haven,CT,1463,0.0,0.0,0.0,0.0


In [167]:
url = "https://dynasties.operationsports.com/team-colors.php?sport=ncaa"
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
lst = soup.select('td')

In [168]:
teamColorsDict= {}
for item in lst[1:]:
    team = item.select('h3')[0].text
    color_boxes = item.select('.team-color-box')
    colors = [box['style'].strip(';').split(':')[1] for box in color_boxes][::-1]
    teamColorsDict[team] = colors

In [225]:
ct = 0
for row in records.itertuples():
    team_name = row.team_name
    if team_name.startswith('E '):
        team_name2 = team_name.replace('E ','Eastern ' )
    elif team_name.startswith('NC '):
        team_name2 = team_name.replace('NC ','North Carolina ' )
    elif team_name.startswith('S ') and team_name != 'S Illinois':
        team_name2 = team_name.replace('S ','South ' )
    elif team_name.startswith('CS '):
        team_name2 = team_name.replace('CS ','Cal State ' )
    elif team_name.startswith('FL '):
        team_name2 = team_name.replace('FL ','Florida ' )
    elif ('Intl') in team_name:
        team_name2 = team_name.replace('Intl','International' )
    elif team_name.startswith('G '):
        team_name2 = team_name.replace('G ','George ' )
    elif team_name.startswith('Loy Marymount'):
        team_name2 = team_name.replace('Loy Marymount','Loyola Marymount' )
    elif team_name.startswith('Miami'):
        team_name2 = team_name.replace('FL','(FL)' )
        team_name2 = team_name2.replace('OH','(OH)' )
    elif team_name.startswith('Missouri KC'):
        team_name2 = team_name.replace('Missouri ','UM' )
    elif team_name.startswith('Monmouth'):
        team_name2 = team_name.replace('NJ','' )
    elif team_name.startswith('MD E'):
        team_name2 = team_name.replace('MD E','Maryland-Eastern' )
    elif team_name.startswith('MS '):
        team_name2 = team_name.replace('MS ','Mississippi ' )
    elif team_name.startswith('MTSU'):
        team_name2 = team_name.replace('MTSU','Middle Tennessee ' )
    elif team_name.startswith('N '):
        team_name2 = team_name.replace('N ','Northern ' )
    elif ('Ole Miss') in team_name:
        team_name2 = 'Mississippi Rebels'
    elif 'S Illinois' in team_name:
        team_name2 = 'Southern Illinois'
    else:
        team_name2 = team_name
    print(team_name, ' --- ', team_name2)
    for k, colors in teamColorsDict.items():
        if team_name2 in k:
            print(colors)
            for i, val in enumerate(colors, start=1):
                records['color{}'.format(i)][records.team_name == team_name] = val
            
            ct += 1
            break
            
print("Number of teams colors extracted for: ", ct)

Abilene Chr  ---  Abilene Chr
Air Force  ---  Air Force
['#0061AA', '#FFFFFF']
Akron  ---  Akron
['#000E41', '#C29C41', '#FFFFFF']
Alabama  ---  Alabama
['#A80532', '#FFFFFF', '#CACCCE']
Alabama A&M  ---  Alabama A&M
['#750000', '#FFFFFF']
Alabama St  ---  Alabama St
['#FECB09', '#231F20', '#FFFFFF']
Albany NY  ---  Albany NY
Alcorn St  ---  Alcorn St
['#42094A', '#FFFFFF', '#FCF5BA']
American Univ  ---  American Univ
['#C41130', '#143D8D', '#FFFFFF']
Appalachian St  ---  Appalachian St
['#000000', '#FFC520', '#FFFFFF']
Arizona  ---  Arizona
['#003559', '#BB003A', '#FFFFFF']
Arizona St  ---  Arizona St
['#96203D', '#FEC927', '#FFFFFF']
Ark Little Rock  ---  Ark Little Rock
Ark Pine Bluff  ---  Ark Pine Bluff
Arkansas  ---  Arkansas
['#CD1041', '#231F20', '#FFFFFF']
Arkansas St  ---  Arkansas St
['#ED1B2E', '#FFFFFF', '#231F20', '#C8C7C7']
Army  ---  Army
['#D19D00', '#000000', '#FFFFFF']
Auburn  ---  Auburn
['#172240', '#FE3300', '#FFFFFF']
Austin Peay  ---  Austin Peay
['#8E0B0B', '#F

/Users/jared/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


  ---  Brown
['#543019', '#8A1E04', '#FFFFFF', '#000000']
Bryant  ---  Bryant
['#702D07', '#231F20', '#A59474']
Bucknell  ---  Bucknell
['#000064', '#FF7000', '#FFFFFF']
Buffalo  ---  Buffalo
['#0068B4', '#FFFFFF', '#000000']
Butler  ---  Butler
['#0D1361', '#FFFFFF', '#000000']
C Michigan  ---  C Michigan
CS Bakersfield  ---  Cal State Bakersfield
['#005DAB', '#FFD204', '#FFFFFF']
CS Fullerton  ---  Cal State Fullerton
['#003768', '#FFFFFF', '#F68F1E']
CS Northridge  ---  Cal State Northridge
['#CD1041', '#FFFFFF', '#231F20']
CS Sacramento  ---  Cal State Sacramento
Cal Baptist  ---  Cal Baptist
Cal Poly SLO  ---  Cal Poly SLO
California  ---  California
['#003768', '#FFDE30', '#FFFFFF']
Campbell  ---  Campbell
['#000000', '#FF8931', '#FFFFFF']
Canisius  ---  Canisius
['#002859', '#EEB211', '#FFFFFF']
Cent Arkansas  ---  Cent Arkansas
Centenary  ---  Centenary
['#A20B36', '#FFFFFF', '#161515']
Central Conn  ---  Central Conn
['#1B49A2', '#FFFFFF']
Charleston So  ---  Charleston So
['#

NE Omaha  ---  NE Omaha
NJIT  ---  NJIT
Navy  ---  Navy
['#131630', '#FCBA29', '#FFFFFF']
Nebraska  ---  Nebraska
['#F20017', '#FFFFFF']
Nevada  ---  Nevada
['#153E5F', '#FFFFFF']
New Hampshire  ---  New Hampshire
['#004990', '#FFFFFF', '#C3C4C6']
New Mexico  ---  New Mexico
['#000000', '#D41045', '#FFFFFF', '#C4C4C4']
New Mexico St  ---  New Mexico St
['#891216', '#221E1F', '#FFFFFF']
New Orleans  ---  New Orleans
['#2B3986', '#FFFFFF', '#FD1A41', '#000000']
Niagara  ---  Niagara
['#69207E', '#FFFFFF', '#201C1D']
Nicholls St  ---  Nicholls St
['#C41230', '#FFFFFF', '#231F20']
Norfolk St  ---  Norfolk St
['#046546', '#FFFFFF', '#F1C409']
North Alabama  ---  North Alabama
North Carolina  ---  North Carolina
['#98BFE5', '#FFFFFF', '#00275D']
North Dakota  ---  North Dakota
['#702D07', '#FFFFFF', '#E22D33', '#00A26B']
North Florida  ---  North Florida
['#0A548D', '#FFFFFF', '#9F9E9D']
North Texas  ---  North Texas
['#00853D', '#FFFFFF', '#2D2327']
Northeastern  ---  Northeastern
['#CC0001

In [226]:
records.head()

,team_name,wins,losses,w/l_ratio,most_recent_elo,team_city,team_state,team_id,color1,color2,color3,color4
0,Abilene Chr,78,106,0.423913,687.653203,Abilene,TX,1101,0,0,0,0
1,Air Force,243,278,0.466411,668.757677,USAF Academy,CO,1102,#0061AA,#FFFFFF,0,0
2,Akron,364,198,0.647687,793.686137,Akron,OH,1103,#000E41,#C29C41,#FFFFFF,0
3,Alabama,324,238,0.576512,801.734237,Tuscaloosa,AL,1104,#A80532,#FFFFFF,#CACCCE,0
4,Alabama A&M,151,340,0.307536,485.358751,Normal,AL,1105,#750000,#FFFFFF,0,0


In [227]:
missing_colors = records[records.color1 == 0]
print('Missing colors for ', missing_colors.shape[0], 'teams')
display(missing_colors)

Missing colors for  63 teams


,team_name,wins,losses,w/l_ratio,most_recent_elo,team_city,team_state,team_id,color1,color2,color3,color4
0,Abilene Chr,78,106,0.423913,687.653203,Abilene,TX,1101,0,0,0,0
6,Albany NY,295,264,0.527728,594.312954,Albany,NY,1107,0,0,0,0
12,Ark Little Rock,262,265,0.497154,704.600947,Little Rock,AR,1114,0,0,0,0
13,Ark Pine Bluff,172,367,0.319109,456.117652,Pine Bluff,AR,1115,0,0,0,0
19,BYU,397,167,0.703901,848.059116,Provo,UT,1140,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
339,W Michigan,286,265,0.519056,649.613823,Kalamazoo,MI,1444,0,0,0,0
340,W Salem St,26,81,0.242991,554.680460,Winston-Salem,NC,1445,0,0,0,0
341,WI Green Bay,288,245,0.540338,678.137077,Green Bay,WI,1453,0,0,0,0
342,WI Milwaukee,275,265,0.509259,593.566394,Milwaukee,WI,1454,0,0,0,0


In [202]:
records[records.team_name.str.contains('Intl')]

,team_name,wins,losses,w/l_ratio,most_recent_elo,team_city,team_state,team_id,color1,color2,color3,color4
95,Florida Intl,184,333,0.355899,688.790679,Miami,FL,1198,0,0,0,0


In [259]:
colors = pd.read_csv('team_colors.csv', index_col=0)

### create script to use with preferences.tps

In [279]:
for hexcode in colors.color1:
    s = "<color>" + hexcode.upper() + "</color>"
    print(" "*8, s)

         <color>#4E2683</color>
         <color>#0061AA</color>
         <color>#000E41</color>
         <color>#A80532</color>
         <color>#750000</color>
         <color>#FECB09</color>
         <color>#46166B</color>
         <color>#42094A</color>
         <color>#C41130</color>
         <color>#000000</color>
         <color>#003559</color>
         <color>#96203D</color>
         <color>#EEB211</color>
         <color>#EEBB00</color>
         <color>#CD1041</color>
         <color>#ED1B2E</color>
         <color>#D19D00</color>
         <color>#172240</color>
         <color>#8E0B0B</color>
         <color>#002D62</color>
         <color>#DA0000</color>
         <color>#008649</color>
         <color>#2A3F76</color>
         <color>#841B35</color>
         <color>#006F53</color>
         <color>#BD8C00</color>
         <color>#2D4492</color>
         <color>#821E1E</color>
         <color>#DD072E</color>
         <color>#4E2400</color>
         <color>#FF151A</color>
        